<a href="https://colab.research.google.com/github/SamuelXJames/Auto-Music-Seperation/blob/main/models/CUNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @ Author: https://github.com/gabolsgabs/cunet
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, multiply, BatchNormalization, Conv1D, Dropout, Lambda,
    Concatenate, Conv2DTranspose, Dense, LeakyReLU,ZeroPadding2D,Cropping2D)
from tensorflow.keras.optimizers import Adam
import numpy as np
#from models.CUNET_config import config
from CUNET_config import config

def FiLM_simple_layer():
    """multiply scalar to a tensor"""
    def func(args):
        x, gamma, beta = args
        s = list(x.shape)
        s[0] = 1
        # avoid tile with the num of batch -> it is the same for both tensors
        g = tf.tile(tf.expand_dims(tf.expand_dims(gamma, 2), 3), s)
        b = tf.tile(tf.expand_dims(tf.expand_dims(beta, 2), 3), s)
        return tf.add(b, tf.multiply(x, g))
    return Lambda(func)

def slice_tensor(position):
    # Crops (or slices) a Tensor
    def func(x):
        return x[:, :, position]
    return Lambda(func)


def cnn_block(
    x, n_filters, kernel_size, padding, initializer, activation='relu'
):
    for i, (f, p) in enumerate(zip(n_filters, padding)):
        extra = i != 0
        x = Conv1D(f, kernel_size, padding=p, activation=activation,
                   kernel_initializer=initializer)(x)
        if extra:
            x = Dropout(0.5)(x)
            x = BatchNormalization(momentum=0.9, scale=True)(x)
    return x

def cnn_control(n_conditions, n_filters):
    """
    For simple dense control:
        - n_conditions = 6
        - n_filters = [16, 32, 128]
    For complex dense control:
        - n_conditions = 1008
        - n_filters = [16, 32, 64]
    """
    input_conditions = Input(shape=(config.Z_DIM, 1), name = 'Film Input')
    initializer = tf.random_normal_initializer(stddev=0.02)
    cnn = cnn_block(
        input_conditions, n_filters, config.Z_DIM, config.PADDING, initializer
    )
    gammas = Dense(
        n_conditions, input_dim=n_filters[-1], activation=config.ACT_G,
        kernel_initializer=initializer
    )(cnn)
    betas = Dense(
        n_conditions, input_dim=n_filters[-1], activation=config.ACT_B,
        kernel_initializer=initializer
    )(cnn)
    # both = Add()([gammas, betas])
    return input_conditions, gammas, betas

def get_activation(name):
    if name == 'leaky_relu':
        return LeakyReLU(alpha=0.2)
    return tf.keras.activations.get(name)


def u_net_conv_block(
    x, n_filters, initializer, gamma, beta, activation, film_type,
    kernel_size=(5, 5), strides=(2, 2), padding='same'
):
    x = Conv2D(n_filters, kernel_size=kernel_size,  padding=padding,
               strides=strides, kernel_initializer=initializer)(x)
    x = BatchNormalization(momentum=0.9, scale=True)(x)
    x = FiLM_simple_layer()([x, gamma, beta])
    
    x = get_activation(activation)(x)
    return x

def u_net_deconv_block(
    x, x_encod, n_filters, initializer, activation, dropout, skip,
    kernel_size=(5, 5), strides=(2, 2), padding='same'
):
    if skip:
        x = Concatenate(axis=3)([x, x_encod])
    x = Conv2DTranspose(
        n_filters, kernel_size=kernel_size, padding=padding, strides=strides,
        kernel_initializer=initializer)(x)
    x = BatchNormalization(momentum=0.9, scale=True)(x)
    if dropout:
        x = Dropout(0.5)(x)
    x = get_activation(activation)(x)
    return x

def getPaddingShape(input_shape):
  output_shape = [input]
  ax1 = (2**np.ceil(np.log2(input_shape[0]))) - input_shape[0]
  a = int(np.floor(ax1/2))
  b = int(np.ceil(ax1/2))

  ax2 = (2**np.ceil(np.log2(input_shape[1]))) - input_shape[1]
  c = int(np.floor(ax2/2))
  d = int(np.ceil(ax2/2))
  return a,b,c,d

def cunet_model(shape = (334,217,1)):
    inputs = Input(shape=shape)#[512,128,1]
    n_layers = config.N_LAYERS
    a,b,c,d = getPaddingShape(input_shape = shape)
    x = inputs
    x = ZeroPadding2D(((a,b),(c,d)))(x)
    encoder_layers = []
    initializer = tf.random_normal_initializer(stddev=0.02)

    input_conditions, gammas, betas = cnn_control(n_conditions=config.N_CONDITIONS, 
                                                  n_filters=config.N_FILTERS)
    # Encoder
    complex_index = 0
    for i in range(n_layers):
        n_filters = config.FILTERS_LAYER_1 * (2 ** i)
        gamma, beta = slice_tensor(i)(gammas), slice_tensor(i)(betas)
        
        x = u_net_conv_block(
            x, n_filters, initializer, gamma, beta,
            activation=config.ACTIVATION_ENCODER, film_type=config.FILM_TYPE
        )
        encoder_layers.append(x)
    # Decoder
    for i in range(n_layers):
        # parameters each decoder layer
        is_final_block = i == n_layers - 1  # the las layer is different
        # not dropout in the first block and the last two encoder blocks
        dropout = not (i == 0 or i == n_layers - 1 or i == n_layers - 2)
        # for getting the number of filters
        encoder_layer = encoder_layers[n_layers - i - 1]
        skip = i > 0    # not skip in the first encoder block
        if is_final_block:
            n_filters = 1
            activation = config.ACT_LAST
        else:
            n_filters = encoder_layer.get_shape().as_list()[-1] // 2
            activation = config.ACTIVATION_DECODER
        x = u_net_deconv_block(
            x, encoder_layer, n_filters, initializer, activation, dropout, skip
        )
    x = Cropping2D(((a,b), (c,d)))(x)
    outputs = multiply([inputs, x])
    model = Model(inputs=[inputs, input_conditions], outputs=outputs)
    return model